In [1]:
import pandas as pd
from pathlib import Path
from glob import glob
import numpy as np
from datetime import datetime,timedelta
from tqdm.notebook import tqdm

Import and sort all caravela ctd eurec4a files, creating a dataframe from this:

In [3]:
files_list = glob(r'C:\Users\uea5\Beth\Eurec4a\During_eurec4a\Caravela_data_callums_pull\Valeport_miniCTD_01\2020\*'+ '/*miniCTD*')  
files_list.sort()

li = []

for filename in files_list:
    df = pd.read_csv(filename, header=0, sep='\t',index_col=False)
    li.append(df)

foo = pd.concat(li, axis=0)

Parse the timestamp to a datetime format

In [5]:
dt = []
for i in tqdm(foo['PC Timestamp[UTC]']):
    dt.append(datetime.strptime(i, '%Y/%m/%d %H:%M:%S.%f'))
foo['dt'] = dt

In [7]:
foo = foo[(foo.dt >= '2020-01-22 00:00:00.000')] #select data from Caravela's launch onwards

Tidy this up to give us the columns we need, with a reasonable index.

In [16]:
foo = foo.reset_index()
foo = foo.drop(['index','PC Time Zone','PC Timestamp[UTC]' ],axis=1)
foo = foo.drop(['level_0'],axis=1) 

KeyError: "['level_0'] not found in axis"

In [26]:
foo = foo.rename(columns = {'dt':'dt[UTC]'}) # define the timezone of dt

Convert to iso time as this is a universally accepted format

In [30]:
a = []
for i in tqdm(range(0,len(foo['dt[UTC]']))):
    a.append(foo['dt[UTC]'][i].isoformat())
foo['dt_iso[UTC]'] = a

In [36]:
foo = foo.drop(['dt[UTC]'], axis = 1)

In [39]:
foo.to_csv('CARAVELA_CTD.csv',index = None)

In [40]:
baa = pd.read_csv('CARAVELA_CTD.csv')

In [41]:
baa


,Pressure[dBar],Temp[C],Conductivity[mS/cm],dt_iso[UTC]
0,10.387,27.220,56.206,2020-01-22T17:20:06.535000
1,10.282,27.226,56.208,2020-01-22T17:20:07.042000
2,10.300,27.224,56.208,2020-01-22T17:20:07.533000
3,10.349,27.226,56.207,2020-01-22T17:20:08.032000
4,10.240,27.226,56.208,2020-01-22T17:20:08.514000
...,...,...,...,...
5455518,10.605,27.072,51.624,2020-02-24T14:20:08.940000
5455519,10.620,27.075,51.411,2020-02-24T14:20:09.428000
5455520,10.617,27.075,51.187,2020-02-24T14:20:09.944000
5455521,10.604,27.070,51.371,2020-02-24T14:20:10.427000


In [45]:
np.sum(np.isnan(foo['Conductivity[mS/cm]']))

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''